In [109]:
import pandas as pd
import numpy as np
%config IPCompleter.greedy=True


In [209]:
df = pd.read_csv('MyFoodData.csv')
df.columns = ['ID', 'Name', 'Group', 'Carbohydrate', 'Ranking']
min_carb_intake = 10
max_carb_intake = 200
target_avg_rank = 8
fridge_items = 4
df = df[df.Ranking > 1] # removing items with rank less than 1 because in worst case they wouldn't meet avg
df = df[df.Group != 'Fats and Oils'] # this is an ingredient of foods, no one eats this by itself
# sanity check that all inputs are valid

,ID,Name,Group,Carbohydrate,Ranking
0,167512,Pillsbury Golden Layer Buttermilk Biscuits Art...,Baked Foods,41,3
1,167513,Pillsbury Cinnamon Rolls With Icing Refrigerat...,Baked Foods,53,4
2,167514,Kraft Foods Shake N Bake Original Recipe Coati...,Baked Foods,80,5
3,167515,George Weston Bakeries Thomas English Muffins,Baked Foods,46,3
6,167518,Waffle Buttermilk Frozen Ready-To-Heat Microwaved,Baked Foods,44,10
...,...,...,...,...,...
14153,789709,Nutritional Drink Or Shake Ready-To-Drink (Mus...,Beverages,2,10
14154,789710,Nutritional Drink Or Shake Ready-To-Drink Ligh...,Beverages,2,4
14155,789713,Nutritional Drink Or Shake High Protein Ready-...,Beverages,1,4
14156,789715,Nutritional Drink Or Shake High Protein Ready-...,Beverages,1,10


In [233]:
fridge = df.sample(n=fridge_items, replace=False, ignore_index=True)
fridge = fridge.sort_values(by=['Ranking'], ascending=False)
menu = pd.DataFrame(columns = ['ID', 'Name', 'Group', 'Carbohydrate', 'Ranking'])
average_rank = fridge['Ranking'].mean()
total_carbs = fridge['Carbohydrate'].sum()
item_counts = menu['ID'].count()

#initializing a list of beverages 
drinks = fridge[(fridge['Group'] == 'Beverages')]
#removing drinks from the fridge
fridge = fridge[(fridge['Group'] != 'Beverages')]
#you can reduce the search space as you go because there are cases that will not meet the conditions
drinks

,ID,Name,Group,Carbohydrate,Ranking
0,789364,Frozen Coffee Drink Nonfat With Whipped Cream,Beverages,13,8


In [234]:
fridge

,ID,Name,Group,Carbohydrate,Ranking
2,787416,Turnip Greens Cooked Ns As To Form Ns As To Fa...,Vegetables,5,7
3,783081,Vienna Sausages Stewed With Potatoes Puerto Ri...,Meats,13,7
1,167544,Snacks Granola Bars Soft Uncoated Raisin,Snacks,66,2


In [235]:
for idx in drinks.index:
    if drinks.empty:
        #think about a variable to track valid menus
        print('there is no valid menu')
        break
    else:
        menu
    for idx in fridge.index:
         for idx in fridge.index:
                 for idx in fridge.index:
                        print(fridge)

       ID                                               Name       Group  \
2  787416  Turnip Greens Cooked Ns As To Form Ns As To Fa...  Vegetables   
3  783081  Vienna Sausages Stewed With Potatoes Puerto Ri...       Meats   
1  167544           Snacks Granola Bars Soft Uncoated Raisin      Snacks   

   Carbohydrate  Ranking  
2             5        7  
3            13        7  
1            66        2  
       ID                                               Name       Group  \
2  787416  Turnip Greens Cooked Ns As To Form Ns As To Fa...  Vegetables   
3  783081  Vienna Sausages Stewed With Potatoes Puerto Ri...       Meats   
1  167544           Snacks Granola Bars Soft Uncoated Raisin      Snacks   

   Carbohydrate  Ranking  
2             5        7  
3            13        7  
1            66        2  
       ID                                               Name       Group  \
2  787416  Turnip Greens Cooked Ns As To Form Ns As To Fa...  Vegetables   
3  783081  Vienna Saus

In [200]:
if fridge.ID[0] in menu.ID:
    print('yes')
else :
    print('no')

SyntaxError: invalid syntax (4158802666.py, line 1)